In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from plotly import express as px
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets, metrics, neighbors, preprocessing, tree
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.metrics import (accuracy_score, f1_score, precision_score,make_scorer,
                             r2_score, recall_score, confusion_matrix)
from sklearn.model_selection import cross_validate
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.impute import SimpleImputer
import numpy as np

In [2]:
class Paths:
    data_path=Path(r'../data').resolve().absolute()
    data_raw=data_path.joinpath(r'01_raw')
    path_train=data_raw.joinpath(r'train.csv')
    path_test = data_raw.joinpath(r'test.csv')

In [3]:
df_test = pd.read_csv(Paths.path_test,decimal='.')
df_train = pd.read_csv(Paths.path_train , decimal='.')

In [4]:
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [5]:
data_limpia = df_train.drop(['Name','PassengerId'], axis=1)
data_limTest = df_test.drop('Name', axis=1)

In [6]:
datosSinDuplicados = data_limpia.drop_duplicates()

In [35]:
data_limTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
dtypes: float64(6), object(6)
memory usage: 401.1+ KB


In [36]:
X_train_num=datosSinDuplicados.drop(datosSinDuplicados.columns[[0,1,2,3,5,11]], axis=1)
x_train_cat=datosSinDuplicados.drop(datosSinDuplicados.columns[[4,6,7,8,9,10]], axis=1)

X_test_num = data_limTest.drop(data_limTest.columns[[0, 1, 2, 3,4, 6]], axis=1)
x_test_cat = data_limTest.drop(data_limTest.columns[[5, 7, 8, 9, 10]], axis=1)

In [37]:
imp_mean_num = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

X_transform_num_test = imp_mean_num.fit_transform(X_test_num)
X_transform_cat_test = imp_mean_cat.fit_transform(x_test_cat)

In [18]:
X_transform_num=imp_mean_num.fit_transform(X_train_num)
X_transform_cat=imp_mean_cat.fit_transform(x_train_cat)

In [19]:
X_traintTransColumn_num=pd.DataFrame(X_transform_num,columns=X_train_num.columns)
X_traintTransColumn_cat=pd.DataFrame(X_transform_cat,columns=x_train_cat.columns)

In [20]:
X_traint_concat=pd.concat([X_traintTransColumn_num,X_traintTransColumn_cat],axis=1)

In [21]:
X_traint_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8678 entries, 0 to 8677
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8678 non-null   float64
 1   RoomService   8678 non-null   float64
 2   FoodCourt     8678 non-null   float64
 3   ShoppingMall  8678 non-null   float64
 4   Spa           8678 non-null   float64
 5   VRDeck        8678 non-null   float64
 6   HomePlanet    8678 non-null   object 
 7   CryoSleep     8678 non-null   object 
 8   Cabin         8678 non-null   object 
 9   Destination   8678 non-null   object 
 10  VIP           8678 non-null   object 
 11  Transported   8678 non-null   object 
dtypes: float64(6), object(6)
memory usage: 813.7+ KB


In [22]:
X_traint_concat['Transported'] =X_traint_concat['Transported'] .astype('bool')

In [23]:
X_traint_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8678 entries, 0 to 8677
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8678 non-null   float64
 1   RoomService   8678 non-null   float64
 2   FoodCourt     8678 non-null   float64
 3   ShoppingMall  8678 non-null   float64
 4   Spa           8678 non-null   float64
 5   VRDeck        8678 non-null   float64
 6   HomePlanet    8678 non-null   object 
 7   CryoSleep     8678 non-null   object 
 8   Cabin         8678 non-null   object 
 9   Destination   8678 non-null   object 
 10  VIP           8678 non-null   object 
 11  Transported   8678 non-null   bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 754.4+ KB


In [24]:
datosSinDuplicadosTrain = X_traint_concat.drop_duplicates()

In [25]:
datosSinDuplicadosTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8676 entries, 0 to 8677
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8676 non-null   float64
 1   RoomService   8676 non-null   float64
 2   FoodCourt     8676 non-null   float64
 3   ShoppingMall  8676 non-null   float64
 4   Spa           8676 non-null   float64
 5   VRDeck        8676 non-null   float64
 6   HomePlanet    8676 non-null   object 
 7   CryoSleep     8676 non-null   object 
 8   Cabin         8676 non-null   object 
 9   Destination   8676 non-null   object 
 10  VIP           8676 non-null   object 
 11  Transported   8676 non-null   bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 821.8+ KB


In [26]:
oe=OrdinalEncoder()
os =RandomOverSampler()
X_train_df = datosSinDuplicadosTrain.drop( 'Transported', axis=1)
y_train_df = datosSinDuplicadosTrain['Transported']

In [27]:

Xov,yov= os.fit_resample(X_train_df, y_train_df)
X_trans = oe.fit_transform(X_train_df)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_trans, y_train_df, test_size=0.3, random_state=1)

In [31]:
decision_tree_model = DecisionTreeClassifier(max_depth=7).fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)

f1_score1 = f1_score(y_test , y_pred, average='macro')
f1_score1

0.7785128537848732

In [30]:
kFold = model_selection.KFold(n_splits=8)
scoring = 'accuracy'
puntaje= (model_selection.cross_val_score(decision_tree_model, X_train, y_train,  scoring = scoring, cv = kFold))
print (f"( {puntaje.mean()}, {puntaje.std()}")

( 0.7678227498093059, 0.011725283288210778
